### Домашнее задание к теме:   
**Рекомендации на основе содержания**

1. Использовать датасет [MovieLens](https://grouplens.org/datasets/movielens/latest/)  
2. Построить рекомендации (регрессия, предсказываем оценку) на фичах:  
 TF-IDF на тегах и жанрах    
 Средние оценки (+median, variance, etc.) пользователя и фильма  
3. Оценить RMSE на тестовой выборке

In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

In [22]:
links = pd.read_csv('links.csv')
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')
tags = pd.read_csv('tags.csv')

Познакомимся с таблицами в датасете:

In [23]:
links.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


Специфичная информация заложена в данной таблице. Как я понял, это три разных видеоресурса, с помощью которых можно найти конкретный фильм. Всё понятно из описания датасета [MovieLens](https://grouplens.org/datasets/movielens/latest/). Данная таблица особой ценности не представляет, в дальнейшем использовать не будем.

In [24]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [25]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [26]:
tags.head()

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


Посмотрим, сколько строк в каждой из таблиц:

In [27]:
links.shape #Для информации, ниже видно, что размерность совпадает с movies.shape

(9742, 3)

In [28]:
movies.shape

(9742, 3)

In [29]:
ratings.shape

(100836, 4)

In [30]:
tags.shape

(3683, 4)

Естественно, есть различия в количестве строк, но для этого надо подготовить данные.

### Варим фичи

Для построения Content-based recommender system (Контент-ориентированной системы рекомендаций) объединим наши таблицы, кроме links в единую с набором фич:  

$\odot$ **movieId**  
$\odot$ **title**  
$\odot$ **genres_transformed** - набор всех жанров к данному фильму в виде строки слов в нижнем регистре через пробелы  
$\odot$ **tags_transformed** - набор всех тегов от всех пользователей к данному фильму, собранный в строку слов в нижнем регистре через пробелы  
$\odot$ **rating_mean** - средний рейтинг фильма по оценкам всех пользователей  
$\odot$ **rating_median** - срединный уровень оценок фильма всеми пользователями (50% процентили)  
$\odot$ **rating_variance** - отклонение оценок фильма пользователями от среднего  значения    

In [31]:
mvs = movies.copy() #Делаем копию. 

Делаем преобразование жанров, заменяем вертикальную черту на пробел, делаем все буквы маленькими.

In [32]:
def change_string(s):
    return ' '.join(s.lower().replace(' ', '').replace('-', '').split('|'))

In [33]:
mvs['genres_transformed'] = mvs.genres.apply(change_string)
mvs.drop(columns=['genres'], inplace=True)

In [34]:
mvs.head()

,movieId,title,genres_transformed
0,1,Toy Story (1995),adventure animation children comedy fantasy
1,2,Jumanji (1995),adventure children fantasy
2,3,Grumpier Old Men (1995),comedy romance
3,4,Waiting to Exhale (1995),comedy drama romance
4,5,Father of the Bride Part II (1995),comedy


Подготовим теги и добавим к таблице mvs

In [35]:
tags_grouped = tags.groupby('movieId')
tags_temporary = tags_grouped.tag.apply(lambda g: ' '.join(list(g)).lower())
tags_temporary_data = pd.DataFrame()
tags_temporary_data['movieId'] = tags_temporary.index
tags_temporary_data['tags_transformed'] = np.array(tags_temporary)
tags_temporary_data.head()

,movieId,tags_transformed
0,1,pixar pixar fun
1,2,fantasy magic board game robin williams game
2,3,moldy old
3,5,pregnancy remake
4,7,remake


Само объеденение в следующем коде:

In [36]:
mvs = pd.merge(mvs, tags_temporary_data, on='movieId', how='outer')
mvs.head()

,movieId,title,genres_transformed,tags_transformed
0,1,Toy Story (1995),adventure animation children comedy fantasy,pixar pixar fun
1,2,Jumanji (1995),adventure children fantasy,fantasy magic board game robin williams game
2,3,Grumpier Old Men (1995),comedy romance,moldy old
3,4,Waiting to Exhale (1995),comedy drama romance,NaN
4,5,Father of the Bride Part II (1995),comedy,pregnancy remake


Есть отсутствующие теги, заменим их на пустые строки.

In [38]:
mvs.tags_transformed = mvs.tags_transformed.fillna('')
mvs.head()

,movieId,title,genres_transformed,tags_transformed
0,1,Toy Story (1995),adventure animation children comedy fantasy,pixar pixar fun
1,2,Jumanji (1995),adventure children fantasy,fantasy magic board game robin williams game
2,3,Grumpier Old Men (1995),comedy romance,moldy old
3,4,Waiting to Exhale (1995),comedy drama romance,
4,5,Father of the Bride Part II (1995),comedy,pregnancy remake


Теперь у нас нет NaN - значений.

In [39]:
mvs.isnull().any().any()

False

Добавим оценки: mean, median, variance. 

In [40]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


Группируем по индексу фильма, имеющему своё название.

In [41]:
ratings_by_movies = ratings.groupby('movieId')

In [43]:
rmean = ratings_by_movies.mean()[['rating']] #Формируем колонку среднего рейтинга фильма с известным movieId
rmean.rename(columns={'rating': 'rating_mean'}, inplace=True) #Заменяем колонки
rmean.reset_index().head()

,movieId,rating_mean
0,1,3.920930
1,2,3.431818
2,3,3.259615
3,4,2.357143
4,5,3.071429


Снова склеиваем таблицы:

In [45]:
mvs = pd.merge(mvs, rmean, on='movieId', how='outer')
mvs.head()

,movieId,title,genres_transformed,tags_transformed,rating_mean
0,1,Toy Story (1995),adventure animation children comedy fantasy,pixar pixar fun,3.920930
1,2,Jumanji (1995),adventure children fantasy,fantasy magic board game robin williams game,3.431818
2,3,Grumpier Old Men (1995),comedy romance,moldy old,3.259615
3,4,Waiting to Exhale (1995),comedy drama romance,,2.357143
4,5,Father of the Bride Part II (1995),comedy,pregnancy remake,3.071429


Проверим есть ли отсутствующие значения в таблице:

In [46]:
mvs.isnull().any()

movieId               False
title                 False
genres_transformed    False
tags_transformed      False
rating_mean            True
dtype: bool

Заменим отсутствующие значения в столбце среднего рейтинга (rating_mean) на 0.

In [47]:
mvs.rating_mean = mvs.rating_mean.fillna(0)
mvs.head()

,movieId,title,genres_transformed,tags_transformed,rating_mean
0,1,Toy Story (1995),adventure animation children comedy fantasy,pixar pixar fun,3.920930
1,2,Jumanji (1995),adventure children fantasy,fantasy magic board game robin williams game,3.431818
2,3,Grumpier Old Men (1995),comedy romance,moldy old,3.259615
3,4,Waiting to Exhale (1995),comedy drama romance,,2.357143
4,5,Father of the Bride Part II (1995),comedy,pregnancy remake,3.071429


In [48]:
mvs.isnull().any()

movieId               False
title                 False
genres_transformed    False
tags_transformed      False
rating_mean           False
dtype: bool

Выберем целевую переменную (rating_mean)

In [49]:
y = mvs[['rating_mean']]
mvs.drop(columns=['rating_mean'], inplace=True)

In [50]:
y.head()

,rating_mean
0,3.920930
1,3.431818
2,3.259615
3,2.357143
4,3.071429


In [51]:
mvs.head()

,movieId,title,genres_transformed,tags_transformed
0,1,Toy Story (1995),adventure animation children comedy fantasy,pixar pixar fun
1,2,Jumanji (1995),adventure children fantasy,fantasy magic board game robin williams game
2,3,Grumpier Old Men (1995),comedy romance,moldy old
3,4,Waiting to Exhale (1995),comedy drama romance,
4,5,Father of the Bride Part II (1995),comedy,pregnancy remake


---

### На подготовленном датасете проведём train_test_split.

In [52]:
mvs.shape #С чем имеем дело, размер

(9742, 4)

In [54]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(mvs, y, test_size=0.2)

### Предварительно преобразуем наши данные с помощью TfidfTransformer и CountVectorizer

In [55]:
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer

#для тегов модели:
cv_tags = CountVectorizer()
tf_tags = TfidfTransformer()

#для жанров модели:
cv_genres = CountVectorizer()
tf_genres = TfidfTransformer()

[Разница между .fit, .fit_transform() и ... transform()](https://stackoverflow.com/questions/23838056/what-is-the-difference-between-transform-and-fit-transform-in-sklearn)  
Мы знаем или не знаем, что хотим предсказать. Используем разное.

In [56]:
#для тегов:
tags_train_cv = cv_tags.fit_transform(x_train.tags_transformed)
tags_train_tf = tf_tags.fit_transform(tags_train_cv)

#для жанров:
genres_train_cv = cv_genres.fit_transform(x_train.genres_transformed)
genres_train_tf = tf_genres.fit_transform(genres_train_cv)

И для тестовой выборки:

In [63]:
tags_test_cv = cv_tags.transform(x_test.tags_transformed)
tags_test_tf = tf_tags.transform(tags_test_cv)

genres_test_cv = cv_genres.transform(x_test.genres_transformed)
genres_test_tf = tf_genres.transform(genres_test_cv)

### Возьмём регрессию и обучим, посчитаем RMSE на тестовой выборке:

### [LinearRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html)

In [64]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [74]:
#для тегов:
lr_tags = LinearRegression(n_jobs=-1)
lr_tags.fit(tags_train_tf, y_train)
y_pred_tags = lr_tags.predict(tags_test_tf)
np.sqrt(mean_squared_error(y_test, y_pred_tags))

0.8599728194515255

In [72]:
#для жанров:
lr_genres = LinearRegression(n_jobs=-1)
lr_genres.fit(genres_train_tf, y_train)
y_pred_genres = lr_genres.predict(genres_test_tf)
np.sqrt(mean_squared_error(y_test, y_pred_genres))

0.814234340516604